In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score,confusion_matrix
import seaborn as sns
df=pd.read_csv("C:\\Users\\dobil\\OneDrive\\Desktop\\MACHINE-LEARNING\\ensemble\\kc_house_data.csv")
df.head()


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [2]:
df.isna().sum()

id               0
date             0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
dtype: int64

In [3]:
df.dtypes

id                 int64
date              object
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object

In [4]:
df.info

<bound method DataFrame.info of                id             date     price  bedrooms  bathrooms  \
0      7129300520  20141013T000000  221900.0         3       1.00   
1      6414100192  20141209T000000  538000.0         3       2.25   
2      5631500400  20150225T000000  180000.0         2       1.00   
3      2487200875  20141209T000000  604000.0         4       3.00   
4      1954400510  20150218T000000  510000.0         3       2.00   
...           ...              ...       ...       ...        ...   
21608   263000018  20140521T000000  360000.0         3       2.50   
21609  6600060120  20150223T000000  400000.0         4       2.50   
21610  1523300141  20140623T000000  402101.0         2       0.75   
21611   291310100  20150116T000000  400000.0         3       2.50   
21612  1523300157  20141015T000000  325000.0         2       0.75   

       sqft_living  sqft_lot  floors  waterfront  view  ...  grade  \
0             1180      5650     1.0           0     0  ...      7   

In [28]:
# Create binary target
median_price = df["price"].median()
df["price_category"] = (df["price"] > median_price).astype(int)


In [29]:
y = df["price_category"]


In [30]:
numerical_features = df.select_dtypes(include=["int64", "float64"]).columns.tolist()
numerical_features.remove("price")
numerical_features.remove("price_category")


In [31]:
categorical_features = df.select_dtypes(include=["object"]).columns.tolist()


In [32]:
df[numerical_features] = df[numerical_features].fillna(
    df[numerical_features].median()
)


In [33]:
X = df[numerical_features]


In [34]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [35]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_scaled, y_train)

lr_pred = lr.predict(X_test_scaled)
lr_acc = accuracy_score(y_test, lr_pred)


In [37]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)

dt_pred = dt.predict(X_test)
dt_acc = accuracy_score(y_test, dt_pred)


In [38]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(X_train_scaled, y_train)

knn_pred = knn.predict(X_test_scaled)
knn_acc = accuracy_score(y_test, knn_pred)


In [39]:
print("Logistic Regression Accuracy:", lr_acc)
print("Decision Tree Accuracy:", dt_acc)
print("KNN Accuracy:", knn_acc)


Logistic Regression Accuracy: 0.8429331482766598
Decision Tree Accuracy: 0.8635207032153597
KNN Accuracy: 0.87323617857969


In [41]:
train_meta_features = pd.DataFrame({
    "lr": lr.predict(X_train_scaled),
    "dt": dt.predict(X_train),
    "knn": knn.predict(X_train_scaled),
})


In [42]:
meta_model = LogisticRegression()
meta_model.fit(train_meta_features, y_train)


LogisticRegression()

In [44]:
test_meta_features = pd.DataFrame({
    "lr": lr.predict(X_test_scaled),
    "dt": dt.predict(X_test),
    "knn": knn.predict(X_test_scaled),
})


In [45]:
stack_pred = meta_model.predict(test_meta_features)


In [51]:
from sklearn.metrics import confusion_matrix

stack_acc = accuracy_score(y_test, stack_pred)

print("Best Base Model Accuracy :", lr_acc)
print("Stacking Model Accuracy:", stack_acc)


Best Base Model Accuracy : 0.8429331482766598
Stacking Model Accuracy: 0.8635207032153597


In [50]:
print("Random Forest Confusion Matrix:")
print(confusion_matrix(y_test, lr_pred))

print("\nStacking Confusion Matrix:")
print(confusion_matrix(y_test, stack_pred))


Random Forest Confusion Matrix:
[[1833  340]
 [ 339 1811]]

Stacking Confusion Matrix:
[[1873  300]
 [ 290 1860]]
